In [ ]:
                                    #Прогнозирование Prophet
import pandas as pd
from prophet import Prophet

# Загрузка исторических данных о продажах из Excel/CSV-файла
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/Sales.csv',sep=';',low_memory=False)

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2021-01-01')]

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
# Подготовка данных для применения модели Prophet(Продажи, замена , на .)
data['y'] = data['y'].str.replace(',' , '.').astype(float)
# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Группировка данных по магазинам и применение модели Prophet для каждого магазина
for store in data['store'].unique():
    store_data = data[data['store'] == store]
    
    # Создание и обучение модели Prophet для каждого магазина
    model = Prophet()
    model.fit(store_data[['ds','y'] + ['store']])

    # Создание DataFrame для прогноза на N дней вперед
    future = model.make_future_dataframe(periods=40)
    forecast = model.predict(future)
    forecast['store'] = store  # Добавляем информацию о магазине
    
    # Добавление прогноза для текущего магазина к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
all_forecasts.to_excel('forecast_next_month.xlsx', index=False)